# Introduction to Recommender Systems

-----


In this notebook, we introduce the concept of a [Recommender System][rs]. Recommender systems are among the most popular machine learning techniques, and, in general, encompass a number of different types of machine learning algorithms that we have covered including classification, regression, and clustering. Some of the most popular recommender system approaches, however, are different and are fundamentally tied to data management. The classic example of a recommender system, is the market basket analysis, where data mining is performed on purchase transaction logs to first learn what types of objects are bought together (e.g., peanut butter and jelly) in order to, second, predict what someone might be interested in buying once they have selected an item (e.g., when peanut butter has been added to the cart, recommend jelly).

More formally, the data structures that support this type  of analysis are known as frequent sets, since we collect sets of frequently purchased items. The most famous algorithm in this category is the [_a priori_][wap] algorithm, where we use what we have learned from previous transactions to make predictions before (i.e., _a priori_ information) someone completes a transaction. More generally, these algorithms are known as _collaborative filtering_, since the algorithm collaboratively filters through records of many individuals to identify trends. Formally, these types of algorithms are used to make recommendations, hence the name _recommender systems_. In addition to the traditional _market basket_ analysis, these algorithms are also used to make recommendations for video, such as Netflix or Hulu, and audio sites, such as Pandora or Spotify.

In fact, anyone who has shopped online has been exposed to these algorithms. For example, at Amazon, when you are viewing the page for a particular item, you also are presented information on _other items you might be interested in_ and on _what other items people buy instead_. The first case is an example of item-based collaborative filtering, where the results of other items people have purchased together with this item are identified. The second is an example of user-based collaborative filtering, where the results from other, similar users, are used to identify items that might be of interest. Sometimes, however, the [results of this analysis can be problematic][tpg], and one should always be considerate of the impact predictions might have on the end user (this is clearly an area where _big brother_ impacts everyone).

In this notebook, we explore a sample data set, movie reviews, that can be used to learn more about recommender systems. Since Python does not (yet) have a standard implementation of recommender systems, we instead develop a single, user-based collaborative filtering example that uses the movie review data to make recommendations for new movies.

-----
[rs]: https://en.wikipedia.org/wiki/Recommender_system
[wap]: https://en.wikipedia.org/wiki/Apriori_algorithm
[tpg]: http://www.workplaceethicsadvice.com/2012/02/target-sends-coupons-to-pregnant-girl-and-unawares-dad-explode.html

## Table of Contents

[Movie Lens Data](#Movie-Lens-Data)

[Exploratory Data Analysis](#Exploratory-Data-Analysis)

[User Based Collaborative Filtering](#User-Based-Collaborative-Filtering)

- [Cosine Similarity](#Cosine-Similarity)

- [Single User Recommendation](#Single-User-Recommendation)

- [Multiple User Recommendation](#Multiple-User-Recommendation)
-----

Before proceeding with the _Formalism_ section of this Notebook, we first have our standard notebook setup code.

-----

In [1]:
# Set up Notebook

# Standard imports
import numpy as np
import numpy.ma as ma
import pandas as pd

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')

-----

[[Back to TOC]](#Table-of-Contents)

## Movie Lens Data

To begin exploring recommender systems, we will explore the [Movie Lens][ml] data. One problem with recommender systems is that they can use significant resources to make predictions since a large quantity of data is required to make the best predictions. Originally, these algorithms were developed to work with relational database systems, working directly within the database engine for maximal performance. As a result, in this Notebook, we will restrict our analysis to the small Movie Lens data set.

We can grab the latest version of the small Movie Lens data set and use within this notebook. Note, to grab the data for yourself, you can execute the following Unix commands, either in a code cell or at the Unix command prompt.

```bash

wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

unzip ml-latest-small.zip
```

The next few Code cells define our local data directory, and if the movie data does not exist, we download the data and _unzip_ it for later use.

-----

In [2]:
# First we find our HOME directory
home_dir = !echo $HOME

# Define data directory
data_dir = home_dir[0] +'/data/'
ml_data_dir = data_dir + 'ml-latest-small'

In [3]:
%%bash -s "$data_dir"

# Note, we passed in a Python variable above to the Bash script 
# which is then accessed via positional parameter, or $1 in this case.

# First test if file of interest does not exist
if [ ! -e "$1"ml-latest-small ] ; then

    # If it does not exist, we grab the file from the Internet and
    # store it locally in the data directory

    wget -O "$1"ml-latest-small.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

    cd "$1"
    unzip -o "$1"ml-latest-small.zip

else
   echo "Files already exist locally."
fi

Archive:  /home/data_scientist/data/ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/movies.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/tags.csv  


--2017-10-23 17:56:22--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918269 (897K) [application/zip]
Saving to: ‘/home/data_scientist/data/ml-latest-small.zip’

     0K .......... .......... .......... .......... ..........  5% 1005K 1s
    50K .......... .......... .......... .......... .......... 11% 1.44M 1s
   100K .......... .......... .......... .......... .......... 16% 8.99M 0s
   150K .......... .......... .......... .......... .......... 22% 11.5M 0s
   200K .......... .......... .......... .......... .......... 27% 2.61M 0s
   250K .......... .......... .......... .......... .......... 33% 9.97M 0s
   300K .......... .......... .......... .......... .......... 39% 2.65M 0s
   350K .......... .......... .......... .......... .......... 44% 11.1

-----

This data set includes a summary document in a file called `README.txt`. This document contains a summary of this data, which is included verbatim below. You can read the entire file either at the command line or via `%load ml-latest-small/README.txt` in a code cell.

> Summary
> =======

> This dataset (ml-latest-small) describes 5-star rating and free-text
> tagging activity from [MovieLens](http://movielens.org), a movie
> recommendation service. It contains 105339 ratings and 6138 tag
> applications across 10329 movies. These data were created by 668 users
> between April 03, 1996 and January 09, 2016. This dataset was generated
> on January 11, 2016.

> Users were selected at random for inclusion. All selected users had
> rated at least 20 movies. No demographic information is included. Each
> user is represented by an id, and no other information is provided.

> The data are contained in four files, `links.csv`, `movies.csv`,
> `ratings.csv` and `tags.csv`. More details about the contents and use of
> all these files follows.

> This is a *development* dataset. As such, it may change over time and is
> not an appropriate dataset for shared research results. See available
> *benchmark* datasets if that is your intent.

> This and other GroupLens data sets are publicly available for download
> at <http://grouplens.org/datasets/>.

-----

In the following code cells, we use Unix commands to display the contents of the small Movie Lens data and the number of ratings included.

-----
[ml]: http://grouplens.org/datasets/movielens/latest/

In [4]:
# Show contents of small movie data set
!ls -la $ml_data_dir

total 3076
drwxr-xr-x 2 data_scientist users    4096 Oct 17  2016 .
drwxr-xr-x 4 data_scientist users    4096 Oct 23 17:56 ..
-rw-r--r-- 1 data_scientist users  183372 Oct 17  2016 links.csv
-rw-r--r-- 1 data_scientist users  458390 Oct 17  2016 movies.csv
-rw-r--r-- 1 data_scientist users 2438266 Oct 17  2016 ratings.csv
-rw-r--r-- 1 data_scientist users    8364 Oct 17  2016 README.txt
-rw-r--r-- 1 data_scientist users   41902 Oct 17  2016 tags.csv


In [5]:
# Count number of lines in ratings
!wc -l $ml_data_dir/ratings.csv

100005 /home/data_scientist/data/ml-latest-small/ratings.csv


-----

[[Back to TOC]](#Table-of-Contents)

## Exploratory Data Analysis

Before developing our recommendation system, we first explore the Movie Lens data. To simplify this task, we will read the ratings data and the movies data into two separate DataFrames. We then display several rows from each DataFrame, as well as compute and display some basic statistics.

-----

In [6]:
import os

# Define full file paths to files
ratings_file = os.path.join(ml_data_dir, 'ratings.csv')
movies_file = os.path.join(ml_data_dir, 'movies.csv')

# Read data into DataFrames
ratings = pd.read_csv(ratings_file)
movies = pd.read_csv(movies_file)

In [7]:
# Sample five random movies
movies.sample(5)

,movieId,title,genres
6008,31035,Testament (1983),Drama
3093,3869,"Naked Gun 2 1/2: The Smell of Fear, The (1991)",Comedy
7644,80969,Never Let Me Go (2010),Drama|Romance|Sci-Fi
4107,5383,"Hound of the Baskervilles, The (1959)",Crime|Horror|Mystery
8480,107406,Snowpiercer (2013),Action|Drama|Sci-Fi


In [8]:
# Summary statistics for the movies data set
movies.describe()

,movieId
count,9125.000000
mean,31123.291836
std,40782.633604
min,1.000000
25%,2850.000000
50%,6290.000000
75%,56274.000000
max,164979.000000


In [9]:
# Sample five random ratings
ratings.sample(5)

,userId,movieId,rating,timestamp
19547,130,1957,5.0,1138997823
28706,212,2427,4.5,1218955624
25296,185,2033,3.0,1003523404
12375,78,345,4.5,1327288678
93146,620,1196,3.5,1455532816


In [10]:
# Summary statistics for the ratings data set
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [11]:
# Number of unique movies in the ratings data file
len(pd.unique(ratings['movieId'].ravel()))

9066

-----

As the previous cells indicated, the `ratings` and `movies` DataFrames have a common column, `movieId`, which we can use to join these DataFrames together. This allows us to better understand these data, since we can see the name of the movie being rated (as opposed to just a `movieId`. In the next few code cells, we join these two DataFrames, display a few rows from the new DataFrame, and use Pandas functionality to display the most popular (by number of reviews) movies, as well as the movies that have the highest average ratings. 

The last task employs several useful tools. First, we perform a `groupby` operation where we employ two operations: mean value and size of sample. This creates a new data structure that has a special column that contains the number of reviews (from the size function) and a special column that contains the average rating (from the `mean` function). We can display the most popular (in terms of average rating), by first restricting the data structure to only those movies that have been rated twenty or more times, and display the first few rows of the sorted data in descending order.

-----

In [12]:
# Merge two dataframes, the common column is selected aotumatically
mv_lens = pd.merge(movies, ratings)

In [13]:
# Display first five lines of combined DataFrame
mv_lens.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [14]:
# Display the most commonly rated movies
mv_lens.title.value_counts().head()

Forrest Gump (1994)                          341
Pulp Fiction (1994)                          324
Shawshank Redemption, The (1994)             311
Silence of the Lambs, The (1991)             304
Star Wars: Episode IV - A New Hope (1977)    291
Name: title, dtype: int64

In [15]:
# Make a new Data structure that holds the movie, number of ratings, and the average rating.
mv_stats = mv_lens.groupby('title').agg({'rating': [np.size, np.mean]})

# Number of ratings to consider top movie
rating_count = 20

# Display most popular movies.
top_movies = mv_stats['rating']['size'] >= rating_count
mv_stats[top_movies].sort_values(by=('rating', 'mean'), ascending=False).head()

rating          
                                   size      mean
title                                            
Godfather, The (1972)             200.0  4.487500
Shawshank Redemption, The (1994)  311.0  4.487138
On the Waterfront (1954)           29.0  4.448276
All About Eve (1950)               38.0  4.434211
Ran (1985)                         26.0  4.423077

-----

[[Back to TOC]](#Table-of-Contents)

## User Based Collaborative Filtering

We can now turn our attention to developing a collaborative filter to make recommendations. The basic idea we will employ is to find the user who is most _similar_ to the current user, and use the ratings from this one similar user to make recommendations. To do this, we need two things. First, we need a matrix that contains the movie ratings, indexed by the user id and the movie id. This allows us to find movies to recommend, given a particular user. Second, we need a definition of similarity, which implies a [distance measurement][scdm] (more similar things should be close, while different things should be far apart). There are a number of different distance measures that are employed, including [Euclidean distance][wed], [Manhattan distance][wmd], and [cosine similarity][wcs].

In this Notebook, we will employ cosine similarity. However, we will not use the implementation in the scipy library since it is not exactly appropriate for what we need. The cosine similarity treats two sets of data as vectors. For example, we can make a vector for each user where each element in the array corresponds to a rating for a specific movie. obviously, for a rating system with many movies, these vectors are extremely long (in our case, there are $10, 325$ elements). The cosine similarity is calculated by multiplying these two vectors and dividing by their length. Thus, this measurement will be scaled to lie between $-1$ and $1$. If the value is one, the vectors are identical. If the value is minus one, the vectors are exactly opposite of each other. And if the value is zero, the vectors are perpendicular to each other. As a result, the cosine similarity is measuring the angle between the two vectors.

To get started, we first construct a pivot table to reference the movie ratings from each user.

-----
[scdm]: http://docs.scipy.org/doc/scipy/reference/spatial.distance.html

[wed]: https://en.wikipedia.org/wiki/Euclidean_distance
[wmd]: https://en.wikipedia.org/wiki/Taxicab_geometry
[wcs]: https://en.wikipedia.org/wiki/Cosine_similarity

In [16]:
# Make a pivot table containing ratings indexed by user id and movie id
tmp_df = ratings.pivot(index='userId', columns='movieId', values='rating')

-----

The values in this pivot table hold the actual ratings. For simplicity, we can restrict our analysis to only _favorable_ ratings, which, since the movies are rated on a five-star system, we take to mean ratings greater than three. We convert the pivot table to hold one for favorable ratings and zero for unfavorable ratings and convert the result to a NumPy matrix. The shape of the matrix indicates we have $671$ unique users, who have each rated one or more of $9,066$ movies.

-----

In [17]:

the_data = tmp_df.applymap(lambda x: 1 if x > 3 else 0).as_matrix()
print(the_data.shape)

(671, 9066)


-----

This matrix is quite large, if we were processing the full movie data set, it would be prohibitively expensive. Fortunately, there is a simple fact that we can leverage. In general, we will not want to include items that haven't been rated enough times. This concept is sometimes called the _support_. Without sufficient ratings, we can't make accurate predictions as the power of this algorithm comes from leveraging data collected from multiple people who (ideally at least) have rated many things.

As a result, we will generate a new pivot table, by first grouping the movies together, sorting into descending order, and selecting only those movies that have been reviewed by multiple people (in this case we will select only movies that have been reviewed at least `rating_count` or more times. By default in this notebook, this variable is twenty, which means only those movies rated at least twenty times will be included in our analysis. Next, we create our pivot table as before, and change the ratings into either zero (if rated three or less) or one (if rated four or five). The shape of the matrix is now considerably smaller, with only $136$ unique users and $928$ unique movies.

-----

In [18]:
# Group ratings by movie ID
mvrs = ratings.groupby('movieId').size().sort_values(ascending=False)

# Select top rated movies
tmp_ratings = ratings.ix[mvrs[mvrs > rating_count].index].dropna()

In [19]:
# Create pivot table
tmp_df = tmp_ratings.pivot(index='userId', columns='movieId', values='rating')

In [20]:
#  Map number of ratings to 0 or 1
the_data = tmp_df.applymap(lambda x: 1 if x > 3 else 0).as_matrix()
print(the_data.shape)

(136, 928)


-----

[[Back to TOC]](#Table-of-Contents)

### Cosine Similarity

We now create our function to compute the cosine similarity between our two vectors. We use simple NumPy commands, and subsequently demonstrate it being used on sample vectors.

----

In [21]:
# Define the Cosine Similarity function

def cosine_similarity(u, v):
    return(np.dot(u, v)/np.sqrt((np.dot(u, u) * np.dot(v, v))))

In [22]:
# Test Cosine Similarity
a = np.array([1, 1, 1, 0, 0])
b = np.array([0, 0, 0, 1, 1])
c = np.array([0, 1, 0, 1, 1])

print('cosine similarity(a, b) = {0:4.3f}'.format(cosine_similarity(a, b)))
print('cosine similarity(a, c) = {0:4.3f}'.format(cosine_similarity(a, c)))
print('cosine similarity(b, c) = {0:4.3f}'.format(cosine_similarity(b, c)))

print('cosine similarity(a, a) = {0:4.3f}'.format(cosine_similarity(a, a)))

cosine similarity(a, b) = 0.000
cosine similarity(a, c) = 0.333
cosine similarity(b, c) = 0.816
cosine similarity(a, a) = 1.000


-----

[[Back to TOC]](#Table-of-Contents)

### Single User Recommendation

We now develop the algorithm to make a recommendation for a single user. To do this, we first create a _fake_ user, by selecting several movies as favorable (effectively we simply make a new user vector). Given this new vector, we compute the cosine similarity between this new user and all users in our reduced data user-movie matrix. We identify the user who is most similar by selecting the row in the user-movie matrix with the highest cosine similarity, extract the movies favorably rated by this particular user, remove any that have already been rated by our _fake_ user, and display the results.

To simplify the identification of the correct movie title, we add a new column to the DataFrame that holds the joined rating and movie data to map between movieId in our user-movie matrix and the movieID used in the original data.

----

In [23]:
# The user-movie matrix
x = the_data

# Make a fake user (with movie ratings that will gaurantee a match)
y = np.zeros(the_data.shape[1], dtype=np.int32)
y[6] = 1 ; y[10] = 1; y[15] = 1; y[64] = 1; y[136] = 1
y[180] = 1; y[230] = 1; y[339] = 1; y[622] = 1; y[703] = 1

# Add a special index column to map the row in the x matrix to the userIds
tmp_df.tmp_idx = np.array(range(x.shape[0]))

In [24]:
# Compute similarity, find maximum value
sims = np.apply_along_axis(cosine_similarity, 1, x, y)
mx = np.nanmax(sims)

# Find the best matching user
usr_idx = np.where(sims==mx)[0][0]

# Print the first thirty reviews of test user and matched user.
print(y[:30])
print(x[usr_idx, :30])

print('\nCosine Similarity(y, x[{0:d}]) = {1:4.3f}' \
      .format(usr_idx, cosine_similarity(y, x[usr_idx])), end='\n\n')

# Now we subtract the vectors
# (any negative value is a movie to recommend)
mov_vec = y - x[usr_idx]

# We want a mask aray, so we zero out any recommended movie.
mov_vec[mov_vec >= 0] = 1
mov_vec[mov_vec < 0] = 0

print(mov_vec[:30])

[0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

Cosine Similarity(y, x[39]) = 0.183

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1]


In [25]:
# Print out the number of movies we will recommend.
print('\n{0} Movie Recommendations for User = {1}' \
      .format(mov_vec[mov_vec == 0].shape[0], 
              tmp_df[tmp_df.tmp_idx == usr_idx].index[0]))


2 Movie Recommendations for User = 43.0


In [26]:
# Get the columns (movieIds) for the current user
mov_ids = tmp_df[tmp_df.tmp_idx == usr_idx].columns

In [27]:
# Now make a masked array to find movies to recommend
# values are the movie ids, mask is the movies the most
# similar user liked.

ma_mov_idx = ma.array(mov_ids, mask = mov_vec)
mov_idx = ma_mov_idx[~ma_mov_idx.mask]        

In [28]:
# Now make a DataFrame of the moves of interest and display

mv_df = movies.ix[movies.movieId.isin(mov_idx)].dropna()

print(60*'-')

for movie in mv_df.title.values:
    print(movie)

print(60*'-', end='\n\n')

------------------------------------------------------------
Braveheart (1995)
Jurassic Park (1993)
------------------------------------------------------------



-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we used cosine similarity to find the user most similar to a test (or fake) user in order to make recommendations. Now that you have run the Notebook, go back and make the following changes to see how the results change.

1. Make a new fake user, ensure that there are movies in common with more than one real user. What movies are now recommended? Do they make sense?
2. Try changing the `ratings_count` variable higher and lower. How does this change the results of the recommendation? Note this affects both the construction of the user-movie DataFrame and the final matrix.
3. The current algorithm uses the only favorable ratings, but that isn't necessary. Try mapping the ratings to the space $-1, 1$ and repeating the analysis. In this mapping, $-1$ corresponds to an unfavorable rating of one. How do the results change?

Finally, the current algorithm finds the _best matching_ user and uses that user's ratings to make a recommendation. In reality, we would want to use $n$ best matching users to make a recommendation. Change the algorithm to capture the five most similar users and to display the aggregated results.

-----

[[Back to TOC]](#Table-of-Contents)

### Multiple User Recommendation

The previous example provided recommendations for a single user. We can extend this example to make recommendations for multiple users. In this case, however, we still use the same basic algorithm. First, we divide our data into two samples: training and testing. We then iterate through our testing set to find the best matching user to compute recommendations for the current testing sample user. Finally, the recommendations are displayed for each user in the test sample.

-----

In [29]:
from sklearn.model_selection import train_test_split

#Define multiple user data set
x, y = the_data, range(the_data.shape[0])

# Create train:test splits
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size = 0.1, random_state=23)

# Add an index into the user-movie DataFrame for the movies that are in the
# user-movie matrix.
tmp_df.tmp_idx = np.array(y)

In [30]:
# Iterate through each user in test set.
for idx, user in enumerate(x_test):
    
    # Compute similarity, find maximum value
    sims = np.apply_along_axis(cosine_similarity, 1, x_train, user)
    mx = np.nanmax(sims)
    
    # If maximum value is a real value    
    if mx > 0:
        
        # Find the index in the similarity matrix with maximum value
        train_idx = np.where(sims==mx)[0][0]
        
        # Now we subtract the vectors 
        # (any negative value is a movie to recommend)
        mov_vec = user - x_train[train_idx]
        
        # We make a mask aray, so we zero out any recommended movie.
        mov_vec[mov_vec >= 0] = 1
        mov_vec[mov_vec < 0] = 0
        
        # We use the fact that y_train has the indices into the original
        # temporary data frame

        user_idx = tmp_df[tmp_df.tmp_idx == y_train[train_idx]]

        # State how many movies are being recommend for this user id
        print('{0} Movie Recommendations for User = {1}' \
              .format(mov_vec[mov_vec == 0].shape[0], \
                      tmp_df[tmp_df.tmp_idx == y_test[idx]].index[0]))
        
        print(60*'-')
        # Now make a masked array to find movies to recommend
        # values are the movie ids, mask is the movies the most
        # similar user liked.
        ma_mov_idx = ma.array(user_idx.columns, mask = mov_vec)
        mov_idx = ma_mov_idx[~ma_mov_idx.mask]
        
        # Now make a DataFrame of the moves of interest and display
        mv_df = movies.ix[movies.movieId.isin(mov_idx)].dropna()
        for movie in mv_df.title.values:
            print(movie)
            
        print(60*'-', end='\n\n')

1 Movie Recommendations for User = 21.0
------------------------------------------------------------
Incredible Hulk, The (2008)
------------------------------------------------------------

0 Movie Recommendations for User = 4.0
------------------------------------------------------------
------------------------------------------------------------

7 Movie Recommendations for User = 481.0
------------------------------------------------------------
Pinocchio (1940)
Lawrence of Arabia (1962)
Beetlejuice (1988)
Planet of the Apes (1968)
Harry Potter and the Prisoner of Azkaban (2004)
Spider-Man 2 (2004)
Shaun of the Dead (2004)
------------------------------------------------------------

34 Movie Recommendations for User = 407.0
------------------------------------------------------------
Casino (1995)
Powder (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Babe (1995)
Usual Suspects, The (1995)
Happy Gilmore (1996)
Braveheart (1995)
Apollo 13 (1995)
Beauty of the Day (Belle de jour) 

------

<font color='red' size = '5'> Student Exercise </font>

In the preceding cell, we computed the cosine similarity between test and training data to make recommendations.  Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Right now, the algorithm uses **only** the best matching user. Change the algorithm to use the five best matching users to develop a more complete set of movie ratings.

2. In some cases, the best matching user does not exist. Change the `rating_count` variable so that all movies are included. How does this change the performance and results of the algorithm?

This example has been a user-based collaborative filtering. If we transpose our matrix, however, we have a movie-user matrix, which can be used to make item-based collaborative filtering recommendations. In this case, you will find users who have rated the current movie and use their other ratings to find movies to recommend. Implement this new algorithm by following this description and the previous code.

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. A nice introduction to [recommendation systems][1] from IBM
2. Wikipedia article on [Recommender system][wr]
3. Wikipedia article on [Apriori algorithm][wa]
4. Wikipedia article on [Cosine similarity][wc]
5. An article on [recommender systems][2] from the Encyclopedia of Machine Learning
6. Article on [association rules][3]
78. Guide to building a [recommender system][rsp] in Python
98. How to implement a [recommender system][irs] in Python
101. Understanding how a [recommender system][urs] works
321. Guide to [item-based filtering][ibf]

-----

[1]: https://www.ibm.com/developerworks/library/os-recommender1/index.html
[wr]: https://en.wikipedia.org/wiki/Recommender_system
[wa]: https://en.wikipedia.org/wiki/Apriori_algorithm
[wc]: https://en.wikipedia.org/wiki/Cosine_similarity
[2]: http://www.prem-melville.com/publications/recommender-systems-eml2010.pdf
[3]: http://aimotion.blogspot.com/2013/01/machine-learning-and-data-mining.html
[rsp]: https://blogs.gartner.com/martin-kihn/how-to-build-a-recommender-system-in-python/
[irs]: https://cambridgespark.com/content/tutorials/implementing-your-own-recommender-systems-in-Python/index.html
[urs]: http://guidetodatamining.com/chapter2/
[ibf]: http://guidetodatamining.com/chapter3/

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode